# Hour Audience
Пример отчета по расчету аудитории интернет-ресурсов (Сross Web) с почасовой разбивкой.

# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt
from mediascope_api.crossweb import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с API Cross Web
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()
cats = cwc.CrossWebCats()

In [ ]:
# вывод возможных статистик для отчета
# cats.get_hour_media_unit().get('statistics')

In [ ]:
# вывод возможных срезов для отчета
# cats.get_hour_media_unit().get('slices')

In [ ]:
# вывод возможных фильтров для отчета
# cats.get_hour_media_unit().get('filters')

## Формируем задания

Необходимо сформировать задание для API Cross Web:
- задать общие параметры для будущего отчета
- расчитать аудиторию интернет-ресурсов с почасовой разбивкой

### Общие параметры для заданий

Для начала зададим общие параметры

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2025-02-01', '2025-02-28')]

# Можно указать период отдельно по будням и выходным:
# БУДНИ
# date_filter = [('2025-01-01', '2025-01-03'), ('2025-01-06', '2025-01-10'), ('2025-01-13', '2025-01-17'), ('2025-01-20', '2025-01-24'), ('2025-01-27', '2025-01-31')]

# ВЫХОДНЫЕ
# date_filter = [('2025-01-04', '2025-01-05'), ('2025-01-11', '2025-01-12'), ('2025-01-18', '2025-01-19'), ('2025-01-25', '2025-01-26')]

# Задаем фильтр по типам пользования интернетом
usetype_filter = [1,2,3]

## Задание

Перейдем к формированию задания


In [ ]:
# Задаем название ресурса для отображения в DataFrame
project_name = 'eventHour'

# Задаем фильтр по географии
geo_filter = None

# Задаем фильтр по демографии
demo_filter = None

# Задаем фильтр по медиа
mart_filter = 'crossMediaResourceId = 1150'

# Задаем почасовой фильтр. Можно указать как группы часов (например, ночные или дневные часы), так и конкретный час:
# hour_filter = 'eventHour > 5'
# Ночные часы: 
#hour_filter = 'eventHour >= 1 AND eventHour <= 6'
# Конкретный час: 
# hour_filter = 'eventHour = 13'
# Если необходимо расчитать весь день:
hour_filter = 'eventHour >= 1 AND eventHour <= 23'

# Указываем список срезов, чтобы сформировать структуру расчета. Можно сделать разбивку по дням, месяцам, кварталам или годам, в зависимости от требований вашего отчета
slices = ["researchMonth", "eventHour"]
# slices = ['researchDate', 'researchMonth', 'researchQuarter', 'researchYear']

# Указываем список статистик для расчета
statistics = ['reach', 'reachPer'] 

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('hour-media', task_name=project_name, date_filter=date_filter,                                
                             geo_filter=geo_filter, demo_filter=demo_filter, 
                             mart_filter=mart_filter, hour_filter=hour_filter,
                             slices=slices, statistics=statistics)

# Отправляем задание на расчет и ждем выполнения
task_audience = mtask.wait_task(mtask.send_task(task_json))

# Получаем результат
df_result = mtask.result2table(mtask.get_result(task_audience), project_name = project_name)
df_result

## Сохраняем в Excel

In [ ]:
writer = pd.ExcelWriter(mtask.get_excel_filename('время'))
df_result.to_excel(writer, 'Report', index=False)
writer.close()